In [1]:
# %load 102_user_smooth.py
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm


# 载入数据
train = load_pickle('../data/train.pkl')
test = load_pickle('../data/test.pkl')
df = load_pickle('../data/df.pkl')

# 贝叶斯平滑参数
iter_num = 10000
epsilon = 0.001

'''
1. 定义需要计算平滑点击率的变量
2. 对于每一天，找出在这之前的所有点击行为
3. 统计该变量的点击次数和购买次数
'''

# smooth_cols = ['user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']
smooth_cols = ['user_gender_id']

# 保存最后结果的dataframe
smooth_train = train[smooth_cols + ['instance_id', 'day']]
smooth_test = test[smooth_cols + ['instance_id','day']]

# 开始对每个特征进行贝叶斯平滑以获取历史交易率
for col in tqdm(smooth_cols):
    # 定义特征名
    col_I = '{}_I'.format(col)
    col_C = '{}_C'.format(col)
    col_smooth_rate = '{}_smooth_rate'.format(col)
    
#     train[col_smooth_rate] = -1
    smooth_all = pd.DataFrame({'day': train.day, '{}'.format(col): train[col]})
    CVR_all = None
    for day in tqdm(range(19, 26)):
        # 统计总浏览数和购买数
        I = train[train.day<day].groupby(col)['is_trade'].count().reset_index()
        I.columns = [col, col_I]
        C = train[train.day<day].groupby(col)['is_trade'].sum().reset_index()
        C.columns = [col, col_C]
        CVR = pd.concat([I, C[col_C]], axis=1)
        
        # CVR的columns：[col, col_I, col_C, 'day']
        CVR['day'] = day
        
        # 贝叶斯平滑过程
        smooth = BayesianSmoothing(1, 1)
        smooth.update(CVR[col_I].values, CVR[col_C].values, iter_num, epsilon)
        alpha = smooth.alpha
        beta = smooth.beta
        CVR[col_smooth_rate] = (CVR[col_C] + alpha) / (CVR[col_I] + alpha + beta)
        
        # 把不同天算的concat起来
        CVR_all = pd.concat([CVR_all, CVR], axis=0)
        # print(CVR.head())
        # smooth_all[col_smooth_rate] = -1
        # print((pd.merge(train[train.day == day], CVR[[col, col_smooth_rate]], on=col, how='inner')).columns[-1])
        # smooth_all[col_smooth_rate][smooth_all.day == day] = (pd.merge(train[train.day == day], CVR[[col, col_smooth_rate]], on=col, how='left')).iloc[:, -1].values

    smooth_train = pd.merge(smooth_train, CVR_all[[col, 'day', col_smooth_rate]], on=[col, 'day'], how='left')
    smooth_test = pd.merge(smooth_test, CVR_all[[col, 'day', col_smooth_rate]], on=[col, 'day'], how='left')


smooth_train.drop(['day', smooth_cols[0]], axis=1, inplace=True)
smooth_test.drop(['day', smooth_cols[0]], axis=1, inplace=True)
print(smooth_train.columns)
print('the shape of train {}'.format(smooth_train.shape))
print('the shape of test {}'.format(smooth_test.shape))
dump_pickle(smooth_train, path='../data/train_feature/102_smooth_features.pkl')
dump_pickle(smooth_test, path='../data/test_feature/102_smooth_features.pkl')








  0%|          | 0/7 [00:00<?, ?it/s]

iter_num: 0
difference of alpha is -0.34616350134938834
difference of beta is 0.11515777903884317
iter_num: 50
difference of alpha is 0.0016961739145337495
difference of beta is 0.04267966087034436
iter_num: 100
difference of alpha is 0.0015383463780433426
difference of beta is 0.04886239574387918
iter_num: 150
difference of alpha is 0.001494210175787436
difference of beta is 0.054221365075898476
iter_num: 200
difference of alpha is 0.0014893077849100766
difference of beta is 0.05888458948003361
iter_num: 250
difference of alpha is 0.0015017593999668843
difference of beta is 0.06297887690388038
iter_num: 300
difference of alpha is 0.0015227398996352504
difference of beta is 0.06660431579108206
iter_num: 350
difference of alpha is 0.001548012970723267
difference of beta is 0.06983892669390457
iter_num: 400
difference of alpha is 0.0015753185821336668
difference of beta is 0.07274422934062841
iter_num: 450
difference of alpha is 0.0016033696574739853
difference of beta is 0.0753693786561

iter_num: 4350
difference of alpha is 0.002171487361859903
difference of beta is 0.10901870233209365
iter_num: 4400
difference of alpha is 0.0021700403279929503
difference of beta is 0.1089283274641275
iter_num: 4450
difference of alpha is 0.002168525587354253
difference of beta is 0.10883463271750315
iter_num: 4500
difference of alpha is 0.0021669453084491863
difference of beta is 0.10873773085245375
iter_num: 4550
difference of alpha is 0.0021653015914715468
difference of beta is 0.1086377309371187
iter_num: 4600
difference of alpha is 0.0021635964708117683
difference of beta is 0.1085347384868669
iter_num: 4650
difference of alpha is 0.0021618319174177003
difference of beta is 0.10842885559674187
iter_num: 4700
difference of alpha is 0.0021600098410825552
difference of beta is 0.10832018107123531
iter_num: 4750
difference of alpha is 0.002158132092578313
difference of beta is 0.10820881054155507
iter_num: 4800
difference of alpha is 0.0021562004657997846
difference of beta is 0.1080


 14%|█▍        | 1/7 [00:00<00:05,  1.09it/s]


iter_num: 7200
difference of alpha is 0.002024966801736028
difference of beta is 0.10082911651863924
iter_num: 7250
difference of alpha is 0.002021792165864511
difference of beta is 0.1006585473440964
iter_num: 7300
difference of alpha is 0.002018609711722519
difference of beta is 0.10048769753268516
iter_num: 7350
difference of alpha is 0.0020154198501725062
difference of beta is 0.10031658700529533
iter_num: 7400
difference of alpha is 0.0020122229808006153
difference of beta is 0.10014523511290463
iter_num: 7450
difference of alpha is 0.002009019492346553
difference of beta is 0.09997366065022106
iter_num: 7500
difference of alpha is 0.002005809762938071
difference of beta is 0.09980188187489603
iter_num: 7550
difference of alpha is 0.002002594160465776
difference of beta is 0.09962991652071196
iter_num: 7600
difference of alpha is 0.001999373042746555
difference of beta is 0.09945778181327114
iter_num: 7650
difference of alpha is 0.001996146757898387
difference of beta is 0.099285

...

iter_num: 4100
difference of alpha is 0.0014006102621939576
difference of beta is 0.07815891307274114
iter_num: 4150
difference of alpha is 0.0014005751378380893
difference of beta is 0.07815707782373238
iter_num: 4200
difference of alpha is 0.001400486625840358
difference of beta is 0.07815214457849606
iter_num: 4250
difference of alpha is 0.0014003458804108249
difference of beta is 0.07814418401221701
iter_num: 4300
difference of alpha is 0.001400154023619038
difference of beta is 0.07813326464849979
iter_num: 4350
difference of alpha is 0.0013999121466312658
difference of beta is 0.07811945295406986
iter_num: 4400
difference of alpha is 0.0013996213109379596
difference of beta is 0.07810281342898406
iter_num: 4450
difference of alpha is 0.0013992825494781869
difference of beta is 0.07808340869013364
iter_num: 4500
difference of alpha is 0.0013988968677400848
difference of beta is 0.078061299551905
iter_num: 4550
difference of alpha is 0.0013984652447813772
difference of beta is 0.07


100%|██████████| 7/7 [00:07<00:00,  1.12s/it]


difference of beta is 0.07604470700647425
iter_num: 6250
difference of alpha is 0.0013620914361744951
difference of beta is 0.07595629723658703
iter_num: 6300
difference of alpha is 0.0013605180634002068
difference of beta is 0.0758665987322047
iter_num: 6350
difference of alpha is 0.0013589222077659002
difference of beta is 0.0757756368722653
iter_num: 6400
difference of alpha is 0.0013573043007575336
difference of beta is 0.0756834365429313
iter_num: 6450
difference of alpha is 0.0013556647658052867
difference of beta is 0.07559002214890143
iter_num: 6500
difference of alpha is 0.0013540040185127111
difference of beta is 0.0754954176285878
iter_num: 6550
difference of alpha is 0.0013523224668183786
difference of beta is 0.07539964646377939
iter_num: 6600
difference of alpha is 0.0013506205112303604
difference of beta is 0.07530273169362545
iter_num: 6650
difference of alpha is 0.0013488985449683355
difference of beta is 0.07520469592435575
iter_num: 6700
difference of alpha is 0.0013

100%|██████████| 1/1 [00:07<00:00,  7.92s/it]

Index(['instance_id', 'user_gender_id_smooth_rate'], dtype='object')
the shape of train (478087, 2)
the shape of test (18371, 2)


In [3]:
smooth_train.isnull().sum()/smooth_train.shape[0]


instance_id                   0.000000
user_gender_id_smooth_rate    0.163693
dtype: float64